In [1]:
#Dependencies 
import pandas as pd 
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from api_keys import g_key
import requests
import json
import time
from bs4 import BeautifulSoup as bs

### NFL ETL AND DATABASE STUFF


In [2]:
##save data URL for nfl attendance, read w/pandas web scraping
data = "https://www.pro-football-reference.com/years/2018/attendance.htm"
nfl_attendance = pd.read_html(data)

In [3]:
#see what columns we have
nfl_attendance = nfl_attendance[0]
for col in nfl_attendance.columns:
    print(col)

Tm
Total
Home
Away
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
Week 9
Week 10
Week 11
Week 12
Week 13
Week 14
Week 15
Week 16
Week 17


In [4]:
#Drop weekly columns, keep total only
nfl_attendance = nfl_attendance.drop([col for col in nfl_attendance.columns if 'Week' in col],axis=1)
nfl_attendance.head()

,Tm,Total,Home,Away
0,Arizona Cardinals,1018127,496111.0,522016.0
1,Atlanta Falcons,1119143,583184.0,535959.0
2,Baltimore Ravens,1053383,563451.0,489932.0
3,Buffalo Bills,1072899,519695.0,553204.0
4,Carolina Panthers,1102756,590182.0,512574.0


In [5]:
#Drop home and away columns, keep team and total attendance only
nfl_attendance = nfl_attendance.drop(["Home", "Away"], axis = 1)
nfl_attendance.head()

,Tm,Total
0,Arizona Cardinals,1018127
1,Atlanta Falcons,1119143
2,Baltimore Ravens,1053383
3,Buffalo Bills,1072899
4,Carolina Panthers,1102756


In [6]:
#Rename columns for easier ETL
nfl_attendance.columns = ["team", "total_attendance"]
nfl_attendance.head()

,team,total_attendance
0,Arizona Cardinals,1018127
1,Atlanta Falcons,1119143
2,Baltimore Ravens,1053383
3,Buffalo Bills,1072899
4,Carolina Panthers,1102756


In [7]:
#This is already in alphabetical order
nfl_attendance

,team,total_attendance
0,Arizona Cardinals,1018127
1,Atlanta Falcons,1119143
2,Baltimore Ravens,1053383
3,Buffalo Bills,1072899
4,Carolina Panthers,1102756
5,Chicago Bears,1045568
6,Cincinnati Bengals,911289
7,Cleveland Browns,1045441
8,Dallas Cowboys,1303393
9,Denver Broncos,1092324


In [8]:
#Data is extracted and ready to be loaded...doesn't need transforming atm
#now do the capacity data! and merge! 
url = "https://www.stadiumsofprofootball.com/comparisons/"
nfl_capacities = pd.read_html(url)
nfl_capacities = nfl_capacities[0]
nfl_capacities

,0,1,2,3,4,5
0,Name,Team(s),Capacity,Opened,Turf,Cost
1,Lambeau Field,Green Bay Packers,80735,9/29/1957,Grass,"$960,000"
2,RingCentral Coliseum,Oakland Raiders,53250,9/18/1966,Grass,$25.5 Million
3,Arrowhead Stadium,Kansas City Chiefs,76416,8/12/1972,Grass,$43 Million
4,New Era Field,Buffalo Bills,73967,8/17/1973,FieldTurf,$22 Million
5,Superdome,New Orleans Saints,76468,9/28/1975,FieldTurf,$134 Million
6,Hard Rock Stadium,Miami Dolphins,65326,8/16/1987,Grass,$115 Million
7,TIAA Bank Field,Jacksonville Jaguars,67264,8/18/1995,Grass,$134 Million
8,Bank of America Stadium,Carolina Panthers,73778,9/14/1996,Grass,$242 Million
9,FedEx Field,Washington Redskins,79000,9/14/1997,Grass,$250 Million


In [9]:
for col in nfl_capacities.columns:
    print(type(col))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [10]:
#Drop unneccessary columns--and stadium name, doesn't matter
nfl_capacities = nfl_capacities.drop([3, 4, 5], axis = 1)
nfl_capacities.head()

,0,1,2
0,Name,Team(s),Capacity
1,Lambeau Field,Green Bay Packers,80735
2,RingCentral Coliseum,Oakland Raiders,53250
3,Arrowhead Stadium,Kansas City Chiefs,76416
4,New Era Field,Buffalo Bills,73967


In [11]:
#Change column names
nfl_capacities.columns = ["stadium", "team", "stadium_capacity"]
nfl_capacities.head()

,stadium,team,stadium_capacity
0,Name,Team(s),Capacity
1,Lambeau Field,Green Bay Packers,80735
2,RingCentral Coliseum,Oakland Raiders,53250
3,Arrowhead Stadium,Kansas City Chiefs,76416
4,New Era Field,Buffalo Bills,73967


In [12]:
#Drop first row, it's repetitive 
nfl_capacities = nfl_capacities.iloc[1:]
nfl_capacities

,stadium,team,stadium_capacity
1,Lambeau Field,Green Bay Packers,80735
2,RingCentral Coliseum,Oakland Raiders,53250
3,Arrowhead Stadium,Kansas City Chiefs,76416
4,New Era Field,Buffalo Bills,73967
5,Superdome,New Orleans Saints,76468
6,Hard Rock Stadium,Miami Dolphins,65326
7,TIAA Bank Field,Jacksonville Jaguars,67264
8,Bank of America Stadium,Carolina Panthers,73778
9,FedEx Field,Washington Redskins,79000
10,M&T Bank Stadium,Baltimore Ravens,71008


In [13]:
#Sort capacity data
nfl_capacities = nfl_capacities.sort_values(by = ['team']).reset_index()
nfl_capacities_cap = nfl_capacities[["stadium_capacity", "stadium"]]
nfl_capacities_cap

,stadium_capacity,stadium
0,63400,University of Phoenix Stadium
1,71000,Mercedes Benz Stadium
2,71008,M&T Bank Stadium
3,73967,New Era Field
4,73778,Bank of America Stadium
5,61500,Soldier Field
6,65515,Paul Brown Stadium
7,68000,FirstEnergy Stadium
8,80000,AT&T Stadium
9,76125,Empower Field


In [14]:
nfl_attendance['stadium_capacity'] = nfl_capacities_cap['stadium_capacity']
nfl_attendance['stadium'] = nfl_capacities_cap['stadium']
nfl_attendance

,team,total_attendance,stadium_capacity,stadium
0,Arizona Cardinals,1018127,63400,University of Phoenix Stadium
1,Atlanta Falcons,1119143,71000,Mercedes Benz Stadium
2,Baltimore Ravens,1053383,71008,M&T Bank Stadium
3,Buffalo Bills,1072899,73967,New Era Field
4,Carolina Panthers,1102756,73778,Bank of America Stadium
5,Chicago Bears,1045568,61500,Soldier Field
6,Cincinnati Bengals,911289,65515,Paul Brown Stadium
7,Cleveland Browns,1045441,68000,FirstEnergy Stadium
8,Dallas Cowboys,1303393,80000,AT&T Stadium
9,Denver Broncos,1092324,76125,Empower Field


In [15]:
nfl_attendance.head()

,team,total_attendance,stadium_capacity,stadium
0,Arizona Cardinals,1018127,63400,University of Phoenix Stadium
1,Atlanta Falcons,1119143,71000,Mercedes Benz Stadium
2,Baltimore Ravens,1053383,71008,M&T Bank Stadium
3,Buffalo Bills,1072899,73967,New Era Field
4,Carolina Panthers,1102756,73778,Bank of America Stadium


In [16]:
#Add empty columns for lat/long 
nfl_attendance["lat"] = ""
nfl_attendance["long"] = ""
nfl_attendance

,team,total_attendance,stadium_capacity,stadium,lat,long
0,Arizona Cardinals,1018127,63400,University of Phoenix Stadium,,
1,Atlanta Falcons,1119143,71000,Mercedes Benz Stadium,,
2,Baltimore Ravens,1053383,71008,M&T Bank Stadium,,
3,Buffalo Bills,1072899,73967,New Era Field,,
4,Carolina Panthers,1102756,73778,Bank of America Stadium,,
5,Chicago Bears,1045568,61500,Soldier Field,,
6,Cincinnati Bengals,911289,65515,Paul Brown Stadium,,
7,Cleveland Browns,1045441,68000,FirstEnergy Stadium,,
8,Dallas Cowboys,1303393,80000,AT&T Stadium,,
9,Denver Broncos,1092324,76125,Empower Field,,


In [17]:
##Get lat/long of each stadium 
params = {"key": g_key}
for index,row in nfl_attendance.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    arena = row["stadium"]
    params["address"]=arena
    result = requests.get(base_url, params = params)
    result = result.json()
    try:
        nfl_attendance.loc[index, "lat"] = result["results"][0]["geometry"]["location"]["lat"]
        nfl_attendance.loc[index, "long"] = result["results"][0]["geometry"]["location"]["lng"]
        time.sleep(1)
    except (IndexError, KeyError):
        print("results not found...skipping")
        time.sleep(1)
print("Complete")

Complete


In [18]:
nfl_attendance

,team,total_attendance,stadium_capacity,stadium,lat,long
0,Arizona Cardinals,1018127,63400,University of Phoenix Stadium,33.5276,-112.263
1,Atlanta Falcons,1119143,71000,Mercedes Benz Stadium,33.7554,-84.4009
2,Baltimore Ravens,1053383,71008,M&T Bank Stadium,39.278,-76.6227
3,Buffalo Bills,1072899,73967,New Era Field,42.7738,-78.787
4,Carolina Panthers,1102756,73778,Bank of America Stadium,35.2258,-80.8528
5,Chicago Bears,1045568,61500,Soldier Field,41.8623,-87.6167
6,Cincinnati Bengals,911289,65515,Paul Brown Stadium,39.0955,-84.5161
7,Cleveland Browns,1045441,68000,FirstEnergy Stadium,41.5061,-81.6995
8,Dallas Cowboys,1303393,80000,AT&T Stadium,32.7473,-97.0945
9,Denver Broncos,1092324,76125,Empower Field,39.7439,-105.02


In [19]:
# A few values in this are wrong--gonna fix them manually
#first, delete last two rows, not enough data to be usable
nfl_attendance_clean = nfl_attendance.iloc[:31]
nfl_attendance_clean.at[27, 'stadium'] = 'Levis Stadium'
nfl_attendance_clean

,team,total_attendance,stadium_capacity,stadium,lat,long
0,Arizona Cardinals,1018127,63400,University of Phoenix Stadium,33.5276,-112.263
1,Atlanta Falcons,1119143,71000,Mercedes Benz Stadium,33.7554,-84.4009
2,Baltimore Ravens,1053383,71008,M&T Bank Stadium,39.278,-76.6227
3,Buffalo Bills,1072899,73967,New Era Field,42.7738,-78.787
4,Carolina Panthers,1102756,73778,Bank of America Stadium,35.2258,-80.8528
5,Chicago Bears,1045568,61500,Soldier Field,41.8623,-87.6167
6,Cincinnati Bengals,911289,65515,Paul Brown Stadium,39.0955,-84.5161
7,Cleveland Browns,1045441,68000,FirstEnergy Stadium,41.5061,-81.6995
8,Dallas Cowboys,1303393,80000,AT&T Stadium,32.7473,-97.0945
9,Denver Broncos,1092324,76125,Empower Field,39.7439,-105.02


In [20]:
nfl_attendance_clean.at[27, 'lat'] = 37.4034    
nfl_attendance_clean.at[27, 'long'] = -121.9694
nfl_attendance_clean

,team,total_attendance,stadium_capacity,stadium,lat,long
0,Arizona Cardinals,1018127,63400,University of Phoenix Stadium,33.5276,-112.263
1,Atlanta Falcons,1119143,71000,Mercedes Benz Stadium,33.7554,-84.4009
2,Baltimore Ravens,1053383,71008,M&T Bank Stadium,39.278,-76.6227
3,Buffalo Bills,1072899,73967,New Era Field,42.7738,-78.787
4,Carolina Panthers,1102756,73778,Bank of America Stadium,35.2258,-80.8528
5,Chicago Bears,1045568,61500,Soldier Field,41.8623,-87.6167
6,Cincinnati Bengals,911289,65515,Paul Brown Stadium,39.0955,-84.5161
7,Cleveland Browns,1045441,68000,FirstEnergy Stadium,41.5061,-81.6995
8,Dallas Cowboys,1303393,80000,AT&T Stadium,32.7473,-97.0945
9,Denver Broncos,1092324,76125,Empower Field,39.7439,-105.02


In [21]:
#Fix steelers row
nfl_attendance_clean.at[26, 'lat'] = 40.4468    
nfl_attendance_clean.at[26, 'long'] = -80.0158
nfl_attendance_clean.at[26, 'stadium'] = 'Heinz Field'
nfl_attendance_clean

,team,total_attendance,stadium_capacity,stadium,lat,long
0,Arizona Cardinals,1018127,63400,University of Phoenix Stadium,33.5276,-112.263
1,Atlanta Falcons,1119143,71000,Mercedes Benz Stadium,33.7554,-84.4009
2,Baltimore Ravens,1053383,71008,M&T Bank Stadium,39.278,-76.6227
3,Buffalo Bills,1072899,73967,New Era Field,42.7738,-78.787
4,Carolina Panthers,1102756,73778,Bank of America Stadium,35.2258,-80.8528
5,Chicago Bears,1045568,61500,Soldier Field,41.8623,-87.6167
6,Cincinnati Bengals,911289,65515,Paul Brown Stadium,39.0955,-84.5161
7,Cleveland Browns,1045441,68000,FirstEnergy Stadium,41.5061,-81.6995
8,Dallas Cowboys,1303393,80000,AT&T Stadium,32.7473,-97.0945
9,Denver Broncos,1092324,76125,Empower Field,39.7439,-105.02


In [22]:
#Fix Eagles row
nfl_attendance_clean.at[25, 'lat'] = 39.9008    
nfl_attendance_clean.at[25, 'long'] = -75.1674
nfl_attendance_clean.at[25, 'stadium'] = 'Lincoln Financial Field'

#Fix Raiders row
nfl_attendance_clean.at[24, 'lat'] = 37.7516    
nfl_attendance_clean.at[24, 'long'] = -122.201
nfl_attendance_clean.at[24, 'stadium'] = 'RingCentral Coliseum'

#Fix Jets row--it is identical to Giants. 
nfl_attendance_clean.at[23, 'lat'] = 40.8135    
nfl_attendance_clean.at[23, 'long'] = -74.0745
nfl_attendance_clean.at[23, 'stadium'] = 'MetLife Stadium'

nfl_attendance_clean

,team,total_attendance,stadium_capacity,stadium,lat,long
0,Arizona Cardinals,1018127,63400,University of Phoenix Stadium,33.5276,-112.263
1,Atlanta Falcons,1119143,71000,Mercedes Benz Stadium,33.7554,-84.4009
2,Baltimore Ravens,1053383,71008,M&T Bank Stadium,39.278,-76.6227
3,Buffalo Bills,1072899,73967,New Era Field,42.7738,-78.787
4,Carolina Panthers,1102756,73778,Bank of America Stadium,35.2258,-80.8528
5,Chicago Bears,1045568,61500,Soldier Field,41.8623,-87.6167
6,Cincinnati Bengals,911289,65515,Paul Brown Stadium,39.0955,-84.5161
7,Cleveland Browns,1045441,68000,FirstEnergy Stadium,41.5061,-81.6995
8,Dallas Cowboys,1303393,80000,AT&T Stadium,32.7473,-97.0945
9,Denver Broncos,1092324,76125,Empower Field,39.7439,-105.02


In [23]:
#Fix Titans row
nfl_attendance_clean.at[30, 'lat'] = 36.1665    
nfl_attendance_clean.at[30, 'long'] = -86.7713
nfl_attendance_clean.at[30, 'stadium'] = 'Nissan Stadium'

#Fix Tampa Bay row
nfl_attendance_clean.at[29, 'lat'] = 27.9759   
nfl_attendance_clean.at[29, 'long'] = -82.5033
nfl_attendance_clean.at[29, 'stadium'] = 'Raymond James Stadium'

#Fix Seahawks
nfl_attendance_clean.at[28, 'lat'] = 47.5952    
nfl_attendance_clean.at[28, 'long'] = -122.3316
nfl_attendance_clean.at[28, 'stadium'] = 'CenturyLink Field'

nfl_attendance_clean

,team,total_attendance,stadium_capacity,stadium,lat,long
0,Arizona Cardinals,1018127,63400,University of Phoenix Stadium,33.5276,-112.263
1,Atlanta Falcons,1119143,71000,Mercedes Benz Stadium,33.7554,-84.4009
2,Baltimore Ravens,1053383,71008,M&T Bank Stadium,39.278,-76.6227
3,Buffalo Bills,1072899,73967,New Era Field,42.7738,-78.787
4,Carolina Panthers,1102756,73778,Bank of America Stadium,35.2258,-80.8528
5,Chicago Bears,1045568,61500,Soldier Field,41.8623,-87.6167
6,Cincinnati Bengals,911289,65515,Paul Brown Stadium,39.0955,-84.5161
7,Cleveland Browns,1045441,68000,FirstEnergy Stadium,41.5061,-81.6995
8,Dallas Cowboys,1303393,80000,AT&T Stadium,32.7473,-97.0945
9,Denver Broncos,1092324,76125,Empower Field,39.7439,-105.02


### Get MLB Data

In [24]:
s = requests.Session()
url = "https://en.wikipedia.org/wiki/2019_Major_League_Baseball_season"

In [25]:
response = requests.Session()
url = "https://en.wikipedia.org/wiki/2019_Major_League_Baseball_season"

In [26]:
response = s.get(url, timeout=10)
response

<Response [200]>

In [27]:

soup = bs(response.text, 'html.parser')

In [28]:
# # testing scraping
title = soup.find(id="firstHeading")
print(title.string)

2019 Major League Baseball season


In [29]:
# Get attendance table
attendance_table = soup.find('table', {'class': 'wikitable sortable'})
# attendance_table()

In [30]:
#More on attendance table?
table=soup.find('table', class_='sortable')
table_rows=table.findAll('tr')
all_data=[]
for each_row in table_rows: 
    try: 
        all_data.append(each_row.findAll('td').text)
    except: 
        pass
all_data

[]

In [31]:
tables=pd.read_html(url)
for idx, each_table in enumerate(tables): 
    print(idx)
    print(each_table.columns)

0
Index(['2019 MLB season', '2019 MLB season.1'], dtype='object')
1
Int64Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')
2
Index(['vteAL East', 'W', 'L', 'Pct.', 'GB', 'Home', 'Road'], dtype='object')
3
Index(['vteAL Central', 'W', 'L', 'Pct.', 'GB', 'Home', 'Road'], dtype='object')
4
Index(['vteAL West', 'W', 'L', 'Pct.', 'GB', 'Home', 'Road'], dtype='object')
5
Index(['vteNL East', 'W', 'L', 'Pct.', 'GB', 'Home', 'Road'], dtype='object')
6
Index(['vteNL Central', 'W', 'L', 'Pct.', 'GB', 'Home', 'Road'], dtype='object')
7
Index(['vteNL West', 'W', 'L', 'Pct.', 'GB', 'Home', 'Road'], dtype='object')
8
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
            19],
           dtype='int64')
9
Index(['Team', 'Former GM', 'Reason for leaving', 'New GM', 'Notes'], dtype='object')
10
Index(['Date', 'Team', 'Former GM', 'Reason for leaving', 'New GM', 'Notes'], dtype='object')
11
Index(['Team', 'Former manager', 'Interim manager', 'Reason for leaving',
       'New 

In [32]:
mlb_attendance = tables[26]

In [33]:
mlb_attendance = pd.DataFrame(mlb_attendance)
TNC = mlb_attendance["Team Name"].str[:-5]
TNC

0       Los Angeles Dodgers
1       St. Louis Cardinals
2          New York Yankees
3              Chicago Cubs
4        Los Angeles Angels
5          Colorado Rockies
6         Milwaukee Brewers
7            Boston Red Sox
8            Houston Astros
9     Philadelphia Phillies
10     San Francisco Giants
11           Atlanta Braves
12            New York Mets
13         San Diego Padres
14          Minnesota Twins
15     Washington Nationals
16     Arizona Diamondbacks
17            Texas Rangers
18          Cincinnati Reds
19         Seattle Mariners
20        Toronto Blue Jays
21        Cleveland Indians
22        Oakland Athletics
23        Chicago White Sox
24           Detroit Tigers
25       Pittsburgh Pirates
26       Kansas City Royals
27        Baltimore Orioles
28           Tampa Bay Rays
29            Miami Marlins
Name: Team Name, dtype: object

In [34]:
mlb_attendance.insert(0,"Team Names",TNC)

In [35]:
mlb_attendance = mlb_attendance[['Team Names','Home attendance', 'Per Game']]
mlb_attendance

,Team Names,Home attendance,Per Game
0,Los Angeles Dodgers,3974309,49066
1,St. Louis Cardinals,3480393,42968
2,New York Yankees,3304404,40795
3,Chicago Cubs,3094865,38208
4,Los Angeles Angels,3023012,37321
5,Colorado Rockies,2993244,36954
6,Milwaukee Brewers,2923333,36091
7,Boston Red Sox,2915502,35994
8,Houston Astros,2857367,35276
9,Philadelphia Phillies,2727421,33672


In [37]:
import csv
mlb_table = pd.read_csv("mlb_attendance.csv")
# mlb_table = mlb_table[['Team Names','Home attendance', 'Per Game', 'Lat', 'Long']]
# mlb_table.drop(["Unnamed: 0"], axis = 1)
del mlb_table["Unnamed: 0"]
mlb_table.columns
# mlb_table
mlb_table

,Team_Names,Home_attendance,Per_Game,Lat,Long
0,Los Angeles Dodgers,3974309,49066,34.072437,-118.246879
1,St. Louis Cardinals,3480393,42968,38.629683,-90.188247
2,New York Yankees,3304404,40795,40.819782,-73.929939
3,Chicago Cubs,3094865,38208,41.947201,-87.656413
4,Los Angeles Angels,3023012,37321,33.799572,-117.889031
5,Colorado Rockies,2993244,36954,39.756980,-104.965329
6,Milwaukee Brewers,2923333,36091,43.042050,-87.905599
7,Boston Red Sox,2915502,35994,42.346613,-71.098817
8,Houston Astros,2857367,35276,29.760450,-95.369784
9,Philadelphia Phillies,2727421,33672,39.952313,-75.162392


### Export NFL and MLB data to SQLite

In [38]:
##Export DF to sqlite

## create engine
engine = create_engine('sqlite:///sports.db', echo = True)
sqlite_connection = engine.connect()

2020-10-18 19:54:50,539 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-18 19:54:50,540 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,541 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-18 19:54:50,543 INFO sqlalchemy.engine.base.Engine ()


In [39]:
#Set name of table to create
nfl_table = "nfl_attendance_clean"
nfl_attendance_clean.to_sql(nfl_table, sqlite_connection, if_exists = 'fail')

2020-10-18 19:54:50,571 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("nfl_attendance_clean")
2020-10-18 19:54:50,572 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,573 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("nfl_attendance_clean")
2020-10-18 19:54:50,574 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,577 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE nfl_attendance_clean (
	"index" BIGINT, 
	team TEXT, 
	total_attendance BIGINT, 
	stadium_capacity TEXT, 
	stadium TEXT, 
	lat FLOAT, 
	long FLOAT
)


2020-10-18 19:54:50,578 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,623 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-18 19:54:50,625 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_nfl_attendance_clean_index ON nfl_attendance_clean ("index")
2020-10-18 19:54:50,625 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,664 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-18 19:54:50,666 INFO sqlalchemy.engine.b

## MLB SQL

In [40]:
#Establish MLB data as own table
mlb_data = "mlb_data"
mlb_table.to_sql(mlb_data, sqlite_connection, if_exists = 'fail')

2020-10-18 19:54:50,717 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("mlb_data")
2020-10-18 19:54:50,718 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,719 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("mlb_data")
2020-10-18 19:54:50,720 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,722 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE mlb_data (
	"index" BIGINT, 
	"Team_Names" TEXT, 
	"Home_attendance" BIGINT, 
	"Per_Game" BIGINT, 
	"Lat" FLOAT, 
	"Long" FLOAT
)


2020-10-18 19:54:50,723 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,776 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-18 19:54:50,778 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_mlb_data_index ON mlb_data ("index")
2020-10-18 19:54:50,778 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,815 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-18 19:54:50,817 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-18 19:54:50,819 INFO sqlalchemy

In [41]:
engine.execute('select * from mlb_data limit 5;').fetchall()

2020-10-18 19:54:50,869 INFO sqlalchemy.engine.base.Engine select * from mlb_data limit 5;
2020-10-18 19:54:50,870 INFO sqlalchemy.engine.base.Engine ()


[(0, 'Los Angeles Dodgers', 3974309, 49066, 34.072437, -118.246879),
 (1, 'St. Louis Cardinals', 3480393, 42968, 38.629683, -90.188247),
 (2, 'New York Yankees', 3304404, 40795, 40.819782000000004, -73.929939),
 (3, 'Chicago Cubs', 3094865, 38208, 41.947201, -87.656413),
 (4, 'Los Angeles Angels', 3023012, 37321, 33.799572, -117.889031)]

In [42]:
#Putting primary key into MLB table
create_query_sql='create table mlb3 (\
id_num bigint primary key,\
Team_Names text,\
Home_attendance	 bigint,\
Per_Game text,\
Lat double,\
Long double\
);'
sqlite_connection.execute(create_query_sql)

2020-10-18 19:54:50,908 INFO sqlalchemy.engine.base.Engine create table mlb3 (id_num bigint primary key,Team_Names text,Home_attendance	 bigint,Per_Game text,Lat double,Long double);
2020-10-18 19:54:50,908 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,949 INFO sqlalchemy.engine.base.Engine COMMIT


In [43]:
sqlite_connection.execute('insert into mlb3 select * from mlb_data')

2020-10-18 19:54:50,969 INFO sqlalchemy.engine.base.Engine insert into mlb3 select * from mlb_data
2020-10-18 19:54:50,970 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:50,975 INFO sqlalchemy.engine.base.Engine COMMIT


In [44]:
#Make fetch happen, check work 
sqlite_connection.execute('select * from mlb3').fetchall()


2020-10-18 19:54:51,036 INFO sqlalchemy.engine.base.Engine select * from mlb3
2020-10-18 19:54:51,037 INFO sqlalchemy.engine.base.Engine ()


[(0, 'Los Angeles Dodgers', 3974309, '49066', 34.072437, -118.246879),
 (1, 'St. Louis Cardinals', 3480393, '42968', 38.629683, -90.188247),
 (2, 'New York Yankees', 3304404, '40795', 40.819782000000004, -73.929939),
 (3, 'Chicago Cubs', 3094865, '38208', 41.947201, -87.656413),
 (4, 'Los Angeles Angels', 3023012, '37321', 33.799572, -117.889031),
 (5, 'Colorado Rockies', 2993244, '36954', 39.75698, -104.965329),
 (6, 'Milwaukee Brewers', 2923333, '36091', 43.04205, -87.905599),
 (7, 'Boston Red Sox', 2915502, '35994', 42.346613, -71.09881700000001),
 (8, 'Houston Astros', 2857367, '35276', 29.76045, -95.369784),
 (9, 'Philadelphia Phillies', 2727421, '33672', 39.952313000000004, -75.16239200000001),
 (10, 'San Francisco Giants', 2707760, '33429', 37.77987, -122.38975400000001),
 (11, 'Atlanta Braves', 2654920, '32777', 33.74691, -84.391239),
 (12, 'New York Mets', 2442532, '30155', 40.75535, -73.84321899999999),
 (13, 'San Diego Padres', 2396399, '29585', 32.752148, -117.143635),
 (14

In [45]:
engine = create_engine('sqlite:///sports.db', echo = True)
sqlite_connection = engine.connect()

2020-10-18 19:54:51,071 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-18 19:54:51,072 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,073 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-18 19:54:51,074 INFO sqlalchemy.engine.base.Engine ()


In [46]:
engine.execute('select * from nfl_attendance_clean limit 5;').fetchall()

2020-10-18 19:54:51,105 INFO sqlalchemy.engine.base.Engine select * from nfl_attendance_clean limit 5;
2020-10-18 19:54:51,106 INFO sqlalchemy.engine.base.Engine ()


[(0, 'Arizona Cardinals', 1018127, '63400', 'University of Phoenix Stadium', 33.5276247, -112.2625593),
 (1, 'Atlanta Falcons', 1119143, '71000', 'Mercedes Benz Stadium', 33.7554491, -84.40085119999999),
 (2, 'Baltimore Ravens', 1053383, '71008', 'M&T Bank Stadium', 39.2779876, -76.6227044),
 (3, 'Buffalo Bills', 1072899, '73967', 'New Era Field', 42.7737546, -78.7869723),
 (4, 'Carolina Panthers', 1102756, '73778', 'Bank of America Stadium', 35.2258108, -80.8528465)]

In [47]:
Base=automap_base()
Base.prepare(sqlite_connection, reflect=True)
Base.classes.keys()

2020-10-18 19:54:51,138 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-10-18 19:54:51,139 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,140 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("mlb3")
2020-10-18 19:54:51,141 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,143 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'mlb3' AND type = 'table'
2020-10-18 19:54:51,144 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,146 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("mlb3")
2020-10-18 19:54:51,147 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,149 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("mlb3")
2020-10-18 19:54:51,150 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,151 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqli

['mlb3', 'nba_table']

In [48]:
sqlite_connection.execute('pragma table_info(nfl_attendance_clean);').fetchall()

2020-10-18 19:54:51,303 INFO sqlalchemy.engine.base.Engine pragma table_info(nfl_attendance_clean);
2020-10-18 19:54:51,304 INFO sqlalchemy.engine.base.Engine ()


[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'team', 'TEXT', 0, None, 0),
 (2, 'total_attendance', 'BIGINT', 0, None, 0),
 (3, 'stadium_capacity', 'TEXT', 0, None, 0),
 (4, 'stadium', 'TEXT', 0, None, 0),
 (5, 'lat', 'FLOAT', 0, None, 0),
 (6, 'long', 'FLOAT', 0, None, 0)]

In [49]:
create_query_sql='create table nfl (\
id_num bigint primary key,\
team text,\
total_attendance bigint,\
stadium_capacity text,\
stadium text,\
lat double,\
long double\
);'
sqlite_connection.execute(create_query_sql)

2020-10-18 19:54:51,338 INFO sqlalchemy.engine.base.Engine create table nfl (id_num bigint primary key,team text,total_attendance bigint,stadium_capacity text,stadium text,lat double,long double);
2020-10-18 19:54:51,339 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,396 INFO sqlalchemy.engine.base.Engine COMMIT


In [50]:
Base=automap_base()
Base.prepare(sqlite_connection, reflect=True)
Base.classes.keys()

2020-10-18 19:54:51,409 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-10-18 19:54:51,410 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,412 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("mlb3")
2020-10-18 19:54:51,413 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,414 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'mlb3' AND type = 'table'
2020-10-18 19:54:51,415 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,417 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("mlb3")
2020-10-18 19:54:51,418 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,419 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("mlb3")
2020-10-18 19:54:51,420 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 19:54:51,420 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqli

['mlb3', 'nba_table', 'nfl']

In [51]:
# sqlite_connection.execute('insert into nfl select * from nfl_attendance_clean')
sqlite_connection.execute('select * from nfl').fetchall()

2020-10-18 19:54:51,562 INFO sqlalchemy.engine.base.Engine select * from nfl
2020-10-18 19:54:51,563 INFO sqlalchemy.engine.base.Engine ()


[]

In [52]:
##Data is now cleaned and extracted & put into its own df. 
nfl_attendance.to_csv("nfl.csv")